In [1]:
import json
import random
import operator
from sklearn.linear_model import RidgeClassifier
import numpy as np
np.random.seed(20)

In [2]:
data = json.load(open('train.json'))
np.random.shuffle(data)

train_data = data[:35000]
validation_data = data[35000:]
test_data = json.load(open('test.json'))
test_ids = [dish['id'] for dish in test_data]

In [3]:
def getTopNPerCat(data,labels,N):
    #Set data structure to hold word counts
    word_count_per_cusine = dict()
    for l in labels:
        word_count_per_cusine[l] = dict()
    
    #Get Word Counts for each ingredient per cuisuine
    for dish in data:
        cuisine = dish['cuisine']
        word_count = word_count_per_cusine[cuisine]
        
        for ing in dish['ingredients']:
            if ing not in word_count:
                word_count[ing] = 1
            else:
                word_count[ing] += 1
    
    
    #Get the top N ingredients per cuisine
    TopN_Per_Cat = dict()
    for l in labels:
        word_count = word_count_per_cusine[l]
        sorted_word_count = sorted(word_count.items(), key=operator.itemgetter(1))
        sorted_word_count.reverse()
        TopN_Per_Cat[l] = sorted_word_count[:N]
    
    #Get all Top N ingredients identified
    all_ings_with_count = []
    for l in labels:
        all_ings_with_count += TopN_Per_Cat[l]
        
    #Remove Count
    all_ings = [ing_count[0] for ing_count in all_ings_with_count]
    
    #Remove Duplicate ingredients
    ing_vector = []
    for item in all_ings:
        if item not in ing_vector:
            ing_vector.append(item)
    
    #Create dict mapping ingredient to index
    ingredient_index = dict()
    for i in  range(0,len(ing_vector)):
        ingredient_index[ing_vector[i]] = i
        
    return ingredient_index

In [11]:
def evaluate(preds, labels):
    corr = 0
    for a,b in zip(preds,labels):
        if a == b:
            corr += 1
    return corr/len(preds)

def writeTest(preds):
    with open('test_preds.csv', 'w') as file:
        writer = csv.writer(file)
        writer.writerow(("id","cuisine"))
        writer.writerows(zip(test_ids,preds))

def getLabels(all_labels):
    labels = set()
    for l in all_labels:
        if l not in labels:
            labels.add(l)

    return labels
  
def getWordCounts(data,label):
    word_count = dict()
    #Get Word Counts for each ingredient per cuisuine
    for dish in data:
        cuisine = dish['cuisine']
        if label == 'all' or label == cuisine:
            for ing in dish['ingredients']:
                if ing not in word_count:
                    word_count[ing] = 1
                else:
                    word_count[ing] += 1
    return word_count

def getFrequency(word_count):
    freqs = dict()
    
    total_count = 0
    for word in word_count:
        total_count += word_count[word]
    
    for word in word_count:
        freqs[word] = word_count[word]/total_count
    
    return freqs

def pickTopNWords(N,freq_overall,freq_cat):
    relative_freqs = dict()
    
    for word in freq_cat:
        relative_freqs[word] = freq_cat[word] - freq_overall[word]
    
    sorted_relative_freqs = sorted(relative_freqs.items(), key=operator.itemgetter(1))
    sorted_relative_freqs.reverse()
    
    TopN = sorted_relative_freqs[:N]
    
    return set([word for (word,freq) in TopN])
    
def assignIndex(lst):
    item_to_index = dict()
    for i in range(0,len(lst)):
        item_to_index[lst[i]] = i
    
    return item_to_index
    
def getIngredients(data):
    ingredients = set()
    for dish in data:
        for ing in dish['ingredients']:
            if ing not in ingredients:
                ingredients.add(ing)
    
    ingredients = list(ingredients)
    
    ingredient_index = dict()
    for i in range(0,len(ingredients)):
        ingredient_index[ingredients[i]] = i
    
    return ingredient_index

def oneHotEncode(data,item_to_index):
    vectors = []
    for dish in data:
        vector = [0]*len(item_to_index)
        ingredients = dish['ingredients']
        for ing in ingredients:
            if ing in item_to_index:
                index = item_to_index[ing]
                vector[index] = 1
        
        vectors.append(vector)
    
    return vectors

In [19]:
trainY = [dish['cuisine'] for dish in train_data]

cuisines = getLabels(trainY)
#ingredient_to_index = getTopNPerCat(train_data,cuisines,100)

#Get Word Counts
overall_word_counts = getWordCounts(train_data,'all')
cuisine_word_counts = dict()
for cuisine in cuisines:
    cuisine_word_counts[cuisine] = getWordCounts(train_data,cuisine)

#Get Frequencies
overall_frequencies = getFrequency(overall_word_counts)
cuisine_frequencies = dict()
for cuisine in cuisines:
    cuisine_frequencies[cuisine] = getFrequency(cuisine_word_counts[cuisine])
    
#Get Top Words
TopWords = set()
for cuisine in cuisines:
    TopWords = TopWords | pickTopNWords(100,overall_frequencies,cuisine_frequencies[cuisine])

#Assign Index
ingredient_to_index = assignIndex(list(TopWords))
#ingredient_to_index = getIngredients(data)

trainX = oneHotEncode(train_data,ingredient_to_index)
#trainY = [dish['cuisine'] for dish in train_data]

validationX = oneHotEncode(validation_data,ingredient_to_index)
validationY = [dish['cuisine'] for dish in validation_data]

testX = oneHotEncode(test_data,ingredient_to_index)

In [20]:
classifier = RidgeClassifier(alpha=1.0)
classifier.fit(trainX, trainY)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)

In [21]:
#Training Evaluation
train_preds = classifier.predict(trainX)
evaluate(train_preds,trainY)

0.7464285714285714

In [22]:
#Validation Evaluation
valid_preds = classifier.predict(validationX)
evaluate(valid_preds,validationY)

0.7276916631755341

In [ ]:
#Write Test Predictions
test_preds = classifier.predict(testX)
writeTest(test_preds)

In [ ]:
cusines = set()
for d in trainY:
    if d not in cusines:
        cusines.add(d)

len(cusines)

# Models TO create
Baseline, Naive Bayes, KNN, Random Forest, Ridge, SVM, Logistic Regression

In [ ]:
labels = cuisines
data = train_data
N = 50

#Set data structure to hold word counts
word_count_per_cusine = dict()
for l in labels:
    word_count_per_cusine[l] = dict()

#Get Word Counts for each ingredient per cuisuine
for dish in data:
    cuisine = dish['cuisine']
    word_count = word_count_per_cusine[cuisine]

    for ing in dish['ingredients']:
        if ing not in word_count:
            word_count[ing] = 1
        else:
            word_count[ing] += 1


#Get the top N ingredients per cuisine
TopN_Per_Cat = dict()
for l in labels:
    word_count = word_count_per_cusine[l]
    sorted_word_count = sorted(word_count.items(), key=operator.itemgetter(1))
    sorted_word_count.reverse()
    TopN_Per_Cat[l] = sorted_word_count[:N]

#Get all Top N ingredients identified
all_ings_with_count = []
for l in labels:
    all_ings_with_count += TopN_Per_Cat[l]

#Remove Count
all_ings = [ing_count[0] for ing_count in all_ings_with_count]

#Remove Duplicate ingredients
ing_vector = []
for item in all_ings:
    if item not in ing_vector:
        ing_vector.append(item)

# #Create dict mapping ingredient to index
# ingredient_index = dict()
# for i in  range(0,len(ing_vector)):
#     ingredient_index[ing_vector[i]] = i



In [15]:
len(TopWords)

515

In [ ]:
s = set([1,2,4])
s2 = set([1,3,4])
s | s2